In [26]:
import m2
import numpy as np
import pickle
from Agent import MaxAgent
import math



In [5]:

f = open('gameplay.pckl', 'rb')
data = pickle.load(f)[1:]
stateActions = []
for start, actions in data:
    for action in actions:
        stateActions.append((start, action))
        start = start.getSuccessor(action)[0]
stateActions
    

[(Cash: 0, (1, 5, 'move')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (0, 0, 'move')),
 (Cash: 26, (3, 9, 'move')),
 (Cash: 26, (0, 9, 'move')),
 (Cash: 47, (40, 45, 'tank')),
 (Cash: 7, (11, 9, 'move')),
 (Cash: 7, (16, 7, 'move')),
 (Cash: 7, (13, 4, 'move')),
 (Cash: 7, (8, 3, 'move')),
 (Cash: 7, (10, 1, 'move')),
 (Cash: 7, (0, 0, 'move')),
 (Cash: 229, (None, None, 'exit')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (1, 5, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 26, (3, 9, 'move')),
 (Cash: 26, (0, 9, 'move')),
 (Cash: 47, (40, 45, 'tank')),
 (Cash: 7, (7, 6, 'move')),
 (Cash: 7, (8, 3, 'move')),
 (Cash: 7, (10, 1, 'move')),
 (Cash: 7, (0, 0, 'move')),
 (Cash: 83, (13, 4, 'move')),
 (Cash: 83, (0, 0, 'move')),
 (Cash: 135, (None, None, 'exit')),
 (Cash: 0, (4, 1, 'move')),
 (Cash: 0, (3, 5, 'move')),
 (Cash: 0, (1, 5, 'move')),
 (Cash: 0, (3, 9, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 57, (40, 45, 'tank')),
 (Cash: 17, (11, 9, 'move')),
 (Cash: 17, (19, 7, 'move')),
 (Cash:

In [6]:
def generateKdata(k):
    data = []
    agent = MaxAgent(depth = k - 1)
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            reward += agent.value(nextState)[0]
            values.append(reward)
        data.append((index, values))
    return data   
                
        
        

In [44]:
def generateQdata(iters):
    data = []
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            _, _, scores, _ = m2.getRollout(nextState, iters)
            values.append(scores[-1])
        data.append((index, values))
    return data   

In [105]:
qdata = generateQdata(100000)

In [87]:
kdata = generateKdata(3)

In [310]:
k3data = kdata

In [311]:
k2data = generateKdata(2)

In [312]:
k1data = generateKdata(1)

In [301]:
def calculate(data, alpha):
    arr = []
    for a, b in data:
        total = 0
        for num in b:
            total += pow(math.e, alpha * num)
        if total == 0:
            arr.append(1 / len(b))
        else:
            arr.append(pow(math.e, alpha * b[a]) / total)
    return np.array(arr, dtype=np.float64)

In [104]:
karr = calculate(kdata, 0.1)

In [110]:
qarr = calculate(qdata, 0.04)

In [158]:
mat = np.column_stack((karr, qarr))

In [174]:
import numpy as np
from scipy.optimize import minimize

m = mat
b = np.array([1 for _ in range(51)])


def loss(x):
    return np.sum(np.square((np.dot(mat, x) - b)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})

res = minimize(loss, x0 =[0,0], method='SLSQP', constraints=cons, options={'disp': True})

print(res.x)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 18.71589702287629
            Iterations: 3
            Function evaluations: 13
            Gradient evaluations: 3
[0.33828651 0.66171349]


In [296]:
def loss(x):
    return -np.sum(np.log(np.dot(mat, x)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0},
       {'type': 'ineq',
         'fun' : lambda x: x[0]},
        {'type': 'ineq',
         'fun' : lambda x: x[1]}
       )

res = minimize(loss, x0 =[0.1,0.9], method='SLSQP', constraints=cons, options={'disp': True})
print(res.x)


# argmax_model P(model | F1 F2 F3 F4 F5 ...)
#  = argmax_model P(model F1 F2 F3 ...)
#  = argmax_model P(F1 F2 F3 ... | model)P(model)
#  = argmax_model P(F1 | model) * .... P(model)
    
#  ~argmax_model P(F1 F2 F3 ... | model)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 51.24533093196085
            Iterations: 4
            Function evaluations: 17
            Gradient evaluations: 4
[0.32878234 0.67121766]


In [315]:
def mix(datas, weights):
    data = []
    for i in range(len(datas[0])):
        total = [0 for _ in range(len(datas[0][i][1]))]
        for j in range(len(datas)):
            for k in range(len(datas[j][i][1])):
                total[k] += weights[j] * datas[j][i][1][k]
        data.append((datas[0][i][0], total))
    return data
#i is sample index, j is dataset index, k is value index
    

In [316]:
def loss(x):
    return np.sum(np.square(calculate(mix([k1data, k2data, k3data, qdata], x[0:4]), x[4]) - b))
#x[0:4] are the linear weights
#x[4] is the alpha value
#mix applies the weights to the valuations of the actions by the different agents and combines them into one
#calculate does the softmax probability calculation based on x[4], the action chosen, and the valuations of each action

b = np.array([1 for _ in range(51)])

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x[0:4]) - 1.0},
       {'type': 'ineq',
         'fun' : lambda x: x[0]},
        {'type': 'ineq',
         'fun' : lambda x: x[1]},
       {'type': 'ineq',
         'fun' : lambda x: x[2]},
        {'type': 'ineq',
         'fun' : lambda x: x[3]})

res = minimize(loss, x0 =[0.25, 0.25, 0.25, 0.25, 0.1], method='SLSQP', constraints=cons, options={'disp': True})

print(res.x)


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 16.838406913715108
            Iterations: 17
            Function evaluations: 127
            Gradient evaluations: 17
[3.70436800e-15 3.75235610e-02 3.31178219e-01 6.31298220e-01
 5.33463555e-01]
